## https://langchain-ai.github.io/langgraph/agents/agents/

In [ ]:
# !pip install -U langgraph "langchain[anthropic]"

In [1]:
from pprint import pprint

In [2]:
# to check the current Python environment
import sys
print(sys.prefix)

import os
print("LANGCHAIN_TRACING_V2:", os.environ.get("LANGCHAIN_TRACING_V2"))

i:\PROGRAMMING PROJECTS\langchain_crash_course\RestaurantNameGenerator\myenv


In [3]:
from dotenv import load_dotenv
import os

# Load variables from .env into environment
load_dotenv()


True

In [4]:
import os
assert "ANTHROPIC_API_KEY" in os.environ
assert "OPENAI_API_KEY" in os.environ


In [9]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [12]:
print(multiply.invoke({"a": 6, "b": 7}))  # returns 42
tool_call = {
    "type": "tool_call",
    "id": "1",
    "args": {"a": 42, "b": 7}
}
multiply.invoke(tool_call)

42


ToolMessage(content='294', name='multiply', tool_call_id='1')

In [27]:
from langgraph.prebuilt import create_react_agent
from langchain_anthropic import ChatAnthropic
from langchain_core.tools import tool
from pprint import pprint

@tool
def multiply(a: float, b: float) -> int:
    """Multiply two numbers."""
    return a * b

@tool
def get_weather(city: str) -> str:  
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

model = ChatAnthropic(
    # model='claude-sonnet-4-20250514',
    model='claude-3-haiku-20240307',
    temperature=0
)

agent = create_react_agent(
    # model='openai:gpt-4o',
    # model='claude-3-5-haiku-latest', # this is the cheapest model available in Anthropic
    # model='claude-sonnet-4-20250514',
    model=model,
    tools=[multiply, get_weather],
    prompt="You are a helpful assistant"
)

response = agent.invoke(
    # {"messages": [{"role": "user", "content": "how is climate in berlin?"}]}
    {"messages": [{"role": "user", "content": "how is 3.14 x 7?"}]}
)
pprint(response)


{'messages': [HumanMessage(content='how is 3.14 x 7?', additional_kwargs={}, response_metadata={}, id='003a4140-3b23-43d5-ae5c-5e2897d5041d'),
              AIMessage(content=[{'id': 'toolu_01F5Eq8Y4K7UZjdvhpSoEBep', 'input': {'a': 3.14, 'b': 7}, 'name': 'multiply', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01W2baqsF6so4Ld7rapoJ7gJ', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 405, 'output_tokens': 70, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-haiku-20240307'}, id='run--3e67e4b8-9c60-4c9d-9fca-e18d6e30ea7b-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.14, 'b': 7}, 'id': 'toolu_01F5Eq8Y4K7UZjdvhpSoEBep', 'type': 'tool_call'}], usage_metadata={'input_tokens': 405, 'output_tokens': 70, 'total_tokens': 475, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}}),
              T

In [1]:
# static prompt that never changes

from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model=model,
    tools=[get_weather],
    # A static prompt that never changes
    prompt="Never answer questions about the weather."
)

agent.invoke(
    {"messages": [{"role": "user", "content": "what is 5x5"}]}
)

ModuleNotFoundError: No module named 'langgraph'

In [31]:
# Dynamic prompt that changes based on the input

from langchain_core.messages import AnyMessage
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.prebuilt import create_react_agent

def prompt(state: AgentState, config: RunnableConfig) -> list[AnyMessage]:  
    user_name = config["configurable"].get("user_name")
    system_msg = f"You are a helpful assistant. Address the user as {user_name}."
    return [{"role": "system", "content": system_msg}] + state["messages"]

agent = create_react_agent(
    model=model,
    tools=[get_weather],
    prompt=prompt
)

agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    config={"configurable": {"user_name": "John Smith"}}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='6d83f6ff-9cc2-4626-9c20-90bbeab67c17'),
  AIMessage(content=[{'text': 'Okay, John Smith, let me check the weather for you in SF:', 'type': 'text'}, {'id': 'toolu_013dso225d8f1b9sVrcE8hS8', 'input': {'city': 'sf'}, 'name': 'get_weather', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01Ly96EeYthgLXKC8anZqWkr', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 345, 'output_tokens': 70, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-haiku-20240307'}, id='run--1083f3f2-b6f8-4967-9563-d23042df8753-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'toolu_013dso225d8f1b9sVrcE8hS8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 345, 'output_tokens': 70, 'total_tokens': 415, 

In [33]:
# Add memory to the agent

from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

agent = create_react_agent(
    model,
    tools=[get_weather],
    checkpointer=checkpointer  
)

# Run the agent
config = {"configurable": {"thread_id": "1"}}
sf_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    config  
)
ny_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what about new york?"}]},
    config
)

In [36]:
#  Configure structured output
# To produce structured responses conforming to a schema, use the response_format parameter. The schema can be defined with a Pydantic model or TypedDict. The result will be accessible via the

from pydantic import BaseModel
from langgraph.prebuilt import create_react_agent

class WeatherResponse(BaseModel):
    conditions: str

agent = create_react_agent(
    model,
    tools=[get_weather],
    response_format=WeatherResponse  
)

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

response["structured_response"]


WeatherResponse(conditions='Sunny and mild')

In [39]:
import sys
print(sys.version)


3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]
